In [ ]:
# 安装python 3.10
!conda create -n venv python=3.10.9 -y

py3 = '/opt/conda/envs/venv/bin/python3'
pip3 = f'{py3} -m pip'

# 安装 torch, torchvision, xformers
!{pip3} install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!{pip3} install https://github.com/Testertime/xformers_P100_precompiled/releases/download/compiled/xformers-0.0.16+bc08bbc.d20230128-cp310-cp310-linux_x86_64.whl

# 其他
!{pip3} install triton tqdm timm fairscale transformers accelerate diffusers albumentations einops safetensors
!{pip3} install bitsandbytes==0.35.0

In [ ]:
root = '/kaggle/working'

# 链接训练样本与模型
# !cp -r /kaggle/input/train-images/image {root}
!ln -s /kaggle/input/chilloutmix-safetensors-models/chilloutmix-Ni-pruned-fp16.safetensors {root}

In [ ]:
# 下载训练脚本
!git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts

# 下载BLIP权重    
!wget 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth'

# 将脚本中library导入环境库中
!touch '{lora_scripts}/sd-scripts/library/__init__.py'
!ln -s '{lora_scripts}/sd-scripts/library/' '/opt/conda/envs/venv/lib/python3.10/site-packages/library'

# 定义常用目录
lora_scripts = f'{root}/lora-scripts'
make_captions_py = f'{lora_scripts}/sd-scripts/finetune/make_captions.py'
train_sh = f'{lora_scripts}/train.sh'

In [ ]:
# 定义一个改写文件内容的函数，方便修改各种脚本
# def sed_i(old_text, new_text, file_path):
#     !sed -i 'g/{old_text}/{new_text}/s' {file_path}
    
from pathlib import Path

def sed_i(old_text, new_text, fp):
    p = Path(fp)
    text = p.read_text()
    text = text.replace(old_text, new_text)
    p.write_text(text)

In [ ]:
# 修改打标图片尺寸
sed_i('IMAGE_SIZE = 384', 'IMAGE_SIZE = 512', make_captions_py)

# 对训练图片打标
train_data_dir = '{root}/image/5_Neilina'

!{py3} {make_captions_py}  {train_data_dir}\
    --caption_weights '{root}/model_large_caption.pth' \
    --batch_size 8

In [ ]:
# 底模路径
sed_i('pretrained_model="./sd-models/model.ckpt"', 
      'pretrained_model="/kaggle/working/chilloutmix-Ni-pruned-fp16.safetensors"', 
      train_sh)

# 训练数据集路径
sed_i('train_data_dir="./train/aki"', 
      'train_data_dir="/kaggle/working/image"', 
      train_sh)

# 输出模型名称
sed_i('output_name="aki"', 'output_name="neilina"', train_sh)

# 使用python 3.10.9 来执行脚本
sed_i('accelerate launch --num_cpu_threads_per_process=8',
      '/opt/conda/envs/venv/bin/python3', 
      train_sh)

# 不使用xformers, 版本不兼容
sed_i('--xformers --shuffle_caption --use_8bit_adam ${extArgs[@]}',
      '--shuffle_caption --use_8bit_adam ${extArgs[@]}',
     train_sh)

!cat {train_sh}

In [ ]:
%cd {lora_scripts}

!bash train.sh

In [108]:
# 压缩输入文件，-m 文件移动到压缩包，-9 压缩率最高
!zip -m -9 outputs.zip {lora_scripts}/output/*

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  adding: kaggle/working/lora-scripts/output/neilina-000002.safetensors (deflated 9%)
  adding: kaggle/working/lora-scripts/output/neilina-000004.safetensors (deflated 9%)
  adding: kaggle/working/lora-scripts/output/neilina-000006.safetensors (deflated 9%)
  adding: kaggle/working/lora-scripts/output/neilina-000008.safetensors (deflated 9%)
  adding: kaggle/working/lora-scripts/output/neilina.safetensors (deflated 9%)


In [114]:
# 解压文件
!unzip -d {root}/output outputs.zip

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Archive:  outputs.zip
  inflating: /kaggle/working/output/kaggle/working/lora-scripts/output/neilina-000002.safetensors  
  inflating: /kaggle/working/output/kaggle/working/lora-scripts/output/neilina-000004.safetensors  
  inflating: /kaggle/working/output/kaggle/working/lora-scripts/output/neilina-000006.safetensors  
  inflating: /kaggle/working/output/kaggle/working/lora-scripts/output/neilina-000008.safetensors  
  inflating: /kaggle/working/output/kaggle/working/lora-scripts/output/neilina.safetensors  
